In [22]:
import re
import os
import spacy
import requests
import json
from tqdm import tqdm

RAW_DATA_PATH = "D:/python/llm_finetune/data/moist_critical_raw_captions/"
PROCESSED_DATA_PATH = "D:/python/llm_finetune/data/processed_captions/"
nlp = spacy.load("en_core_web_sm")

In [14]:
paragraphs = []
for raw_file in os.listdir(RAW_DATA_PATH):
    file_path = os.path.join(RAW_DATA_PATH, raw_file)
    with open(file_path, "r", encoding="utf-8") as f:
        raw_text = f.read()

    doc = nlp(raw_text)
    sentences = [sent.text for sent in doc.sents]

    sent_counter = 0
    paragraph = ""
    for sentence in sentences:
        paragraph += f" {sentence} "
        if sent_counter >= 5:
            paragraphs.append(paragraph)
            paragraph = ""
            sent_counter = 0

        sent_counter += 1

    break

paragraphs

[" All right, you win.  I'll roll over and\nshow you my belly.  I submit.  My arm has\nbeen getting twisted by tons of people\ndemanding that I speak about the Sydney\nSweeney bathwater soap.  Apparently, it\nsold out in seconds.  It's uh taken taken\nthe world by storm as the utmost\nauthority on both bathwater and soap.\n ",
 " People are turning to my expert analysis\nhere to weigh in on this subject.\n  Obviously, I have my own soap, Cheeky,\nwhich is far better than this horse\nhockey.  Cheeky is the best soap in the\nuniverse that we've made.  But don't just\ntake my word for it.  Go to usech\ncheeky.com right now and get some soap\nfor yourself to try it out. ",
 " It'll change\nyour life for the better.  This topic\nisn't meant to just be a vessel to glaze\ncheeky.  It's just because I have that so\nmany people keep emailing about this\nSydney Sweeney stuff, so I had to\nmention it.  It's become extremely clear\nthat Sydney Sweeney has teamed up with\nthese schmucks over there 

In [11]:
def summarize_paragraph(model, paragraph, instruction):

    prompt = f"Instruction: {instruction}\nInput: {paragraph}\nOutput:"
    response = requests.post(
        "http://localhost:11434/api/generate",
        json={
            "model": model,
            "prompt": prompt,
        },
        stream=True,
    )
    output_text = ""
    for line in response.iter_lines():
        if line:
            data = json.loads(line)
            output_text += data.get("response", "")
    return prompt, output_text

In [27]:
instruction = "Remove the author's tone and rewrite the sentence in a simple, factual way like it was in a Wikipedia article. Do not explain your reasoning. Output only the final rewritten text"

# models = ["gemma3:4b", "llama3.2:latest", "phi:latest"]
models = ["deepseek-r1:14b"]

for model in models:
    model_clean_name = model.replace(":", "-")
    results_list = []
    for paragraph in tqdm(paragraphs):
        prompt, output_text = summarize_paragraph(
            model=model, paragraph=paragraph, instruction=instruction
        )

        resultant_dict = {
            "instruction": instruction,
            "input": paragraph,
            "output": output_text,
        }
        results_list.append(resultant_dict)

    if not os.path.exists(os.path.join(PROCESSED_DATA_PATH, model)):
        os.mkdir(os.path.join(PROCESSED_DATA_PATH, model_clean_name))

        with open(
            os.path.join(
                PROCESSED_DATA_PATH,
                model_clean_name,
                f"{model_clean_name}_summary.json",
            ),
            "w",
        ) as f:
            json.dump(results_list, f, indent=4)

100%|██████████| 19/19 [07:23<00:00, 23.36s/it]
